In [33]:
import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import faiss
import time


In [34]:
# Loading all embeddings (which were gotten from SAINT)

path = "../5_embeddings/cls_embeddings_time.struct_time(tm_year=2025, tm_mon=2, tm_mday=7, tm_hour=19, tm_min=40, tm_sec=22, tm_wday=4, tm_yday=38, tm_isdst=0).npy"

cls_embeddings = np.load(path)
print(cls_embeddings.shape)


(118108, 32)


In [35]:
train_df = pd.read_csv("../2_dataset/final/train_df.csv")
train_df

,cls,ProductCD,card4,card6,P_emaildomain,Unnamed: 0,TransactionDT,TransactionAmt,card1,card2,...,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321
0,0,4,2,1,2,-0.291883,-0.329939,0.108390,-0.145421,-0.399322,...,-0.227583,-0.222385,-0.249222,-0.229148,-0.048377,-0.062211,-0.058049,-0.055287,-0.088855,-0.074142
1,0,4,3,2,16,0.892993,0.871243,-0.359702,0.680504,-0.412094,...,-0.030054,-0.222385,-0.249222,-0.229148,-0.048377,-0.062211,-0.058049,-0.055287,-0.088855,-0.074142
2,0,4,2,2,1,-1.594876,-1.467121,8.134522,-0.109308,0.711822,...,-0.227583,-0.222385,-0.249222,-0.229148,-0.048377,-0.062211,-0.058049,-0.055287,-0.088855,-0.074142
3,0,4,3,2,19,-0.123148,-0.156138,-0.422421,1.487250,-0.265218,...,0.341765,-0.222385,-0.249222,-0.229148,-0.048377,-0.062211,-0.058049,-0.055287,-0.088855,-0.074142
4,0,4,3,2,16,1.611964,1.677853,-0.317889,-0.081355,-0.265218,...,-0.227583,-0.222385,-0.249222,-0.229148,-0.048377,-0.062211,-0.058049,-0.055287,-0.088855,-0.074142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76765,0,4,3,2,16,-1.327921,-1.260784,-0.113217,-0.653259,-1.606254,...,1.009878,-0.222385,-0.249222,-0.229148,-0.048377,-0.062211,-0.058049,-0.055287,-0.088855,-0.074142
76766,0,4,2,2,25,0.675641,0.648266,-0.075376,-0.002802,0.756523,...,-0.227583,-0.222385,-0.249222,-0.229148,-0.048377,-0.062211,-0.058049,-0.055287,-0.088855,-0.074142
76767,0,4,1,1,16,0.418154,0.377752,0.150412,-1.485918,-0.226903,...,-0.227583,0.393449,0.090945,0.276274,-0.048377,-0.062211,-0.058049,-0.055287,-0.088855,-0.074142
76768,0,4,3,2,19,0.605578,0.576883,-0.322279,-0.182963,0.577719,...,-0.227583,-0.222385,-0.249222,-0.229148,-0.048377,-0.062211,-0.058049,-0.055287,-0.088855,-0.074142


In [36]:
train_y = pd.read_csv("../2_dataset/final/train_y_df.csv")
train_y.shape

(76770, 1)

In [ ]:
test_df = pd.read_csv("../2_dataset/final/test_df.csv")
test_y = pd.read_csv("../2_dataset/final/test_y_df.csv")
test_y['isFraud'] = test_y['isFraud'].astype(np.float32)

In [38]:
val_df = pd.read_csv("../2_dataset/final/val_df.csv")
val_y = pd.read_csv("../2_dataset/final/val_y_df.csv")

In [39]:
train_y.shape, train_df.shape

((76770, 1), (76770, 182))

In [40]:
test_y.shape, test_df.shape

((23622, 1), (23622, 182))

In [41]:
val_y.shape, val_df.shape

((17716, 1), (17716, 182))

## Data Exploration


In [42]:
train_combined = pd.concat([train_df, train_y], axis=1)
for i in train_combined.columns:
    print(i)


cls
ProductCD
card4
card6
P_emaildomain
Unnamed: 0
TransactionDT
TransactionAmt
card1
card2
card3
card5
addr1
addr2
C1
C2
C3
C4
C5
C6
C7
C8
C9
C10
C11
C12
C13
C14
D1
D10
D15
V12
V13
V14
V15
V16
V17
V18
V19
V20
V21
V22
V23
V24
V25
V26
V27
V28
V29
V30
V31
V32
V33
V34
V53
V54
V55
V56
V57
V58
V59
V60
V61
V62
V63
V64
V65
V66
V67
V68
V69
V70
V71
V72
V73
V74
V75
V76
V77
V78
V79
V80
V81
V82
V83
V84
V85
V86
V87
V88
V89
V90
V91
V92
V93
V94
V95
V96
V97
V98
V99
V100
V101
V102
V103
V104
V105
V106
V107
V108
V109
V110
V111
V112
V113
V114
V115
V116
V117
V118
V119
V120
V121
V122
V123
V124
V125
V126
V127
V128
V129
V130
V131
V132
V133
V134
V135
V136
V137
V279
V280
V281
V282
V283
V284
V285
V286
V287
V288
V289
V290
V291
V292
V293
V294
V295
V296
V297
V298
V299
V300
V301
V302
V303
V304
V305
V306
V307
V308
V309
V310
V311
V312
V313
V314
V315
V316
V317
V318
V319
V320
V321
isFraud


## Correlation Calculation

In [43]:
correlation_matrix = train_combined.corr()

# Extract correlation with target variable 'isFraud'
target_correlation = correlation_matrix["isFraud"].sort_values(ascending=False)

# Plot the heatmap
# plt.figure(figsize=(10, 6))
# plt.barh(target_correlation.index, target_correlation.values, color="skyblue")
# plt.xlabel("Correlation Coefficient")
# plt.ylabel("Features")
# plt.title("Feature Correlation with isFraud")
# plt.grid(True)
# plt.show()

# Display the correlation matrix as a table
target_correlation = correlation_matrix["isFraud"].sort_values(ascending=False)

# Print correlation with target variable
# print("Correlation with isFraud:\n", target_correlation[0:15])

# for i in target_correlation:
# print(i)


# V86          0.238604
# V87          0.233544
# V79          0.178765
# V94          0.172088


In [44]:
train_combined["V86"]
column_range = train_df["V86"].max() - train_df["V86"].min()
# Range of 'V86': 69.6747708 low: -2.7227058  high: 66.952065

In [45]:
train_combined["V86"].unique()

array([-0.14215863,  5.0189357 ,  2.4383886 , 10.18003   ,  7.5994825 ,
       30.824408  , -2.7227058 , 17.92167   , 12.760577  , 28.24386   ,
       20.502218  , 23.082766  , 15.341125  , 35.9855    , 41.146595  ,
       25.663313  , 54.04933   , 51.468784  , 66.952065  ])

In [46]:
train_combined["V94"].unique()

# V86          0.238604
# V87          0.233544
# V79          0.178765
# V94          0.172088


array([-0.3627873,  2.7560682,  5.8749237])

## Splitting Embeddings


In [47]:
# Compute 65% of the total rows
total_rows = cls_embeddings.shape[0]
train_size = int(0.65 * total_rows)  # 65% of 118108
test_size = int(0.8 * total_rows)

# Slice the top 65%
train_embeddings = cls_embeddings[:train_size]
val_embeddings = cls_embeddings[train_size:test_size]
test_embeddings = cls_embeddings[test_size:]

print(f"Total embeddings shape: {cls_embeddings.shape}")
print(f"Train embeddings shape: {train_embeddings.shape}")
print(f"Val embeddings shape: {val_embeddings.shape}")
print(f"Test embeddings shape: {test_embeddings.shape}")

Total embeddings shape: (118108, 32)
Train embeddings shape: (76770, 32)
Val embeddings shape: (17716, 32)
Test embeddings shape: (23622, 32)


#### faiss index and similarity search


In [48]:
def create_index(num_embeddings, dimension):
    # num_embeddings = 76770
    # dimension = 32
    index = faiss.IndexFlatL2(dimension)  # L2 similarity
    index.add(train_embeddings)  # index of pre-computed embeddings

    return index



In [49]:
def search_faiss(query_vector):
    index = create_index(76770, 32)

    # Convert PyTorch tensor to NumPy
    if isinstance(query_vector, torch.Tensor):
        query_vector = query_vector.detach().cpu().numpy()

    query_vector = query_vector.astype("float32").reshape(1, -1)
    query_vector = query_vector.astype("float32").reshape(1, -1)

    # k = 120, As best result for 120
    distances, indices = index.search(
        query_vector, k=120
    )  # by default using euclidean distance for similarity

    indices = indices.flatten()
    return distances, indices

#### L2 distance component


In [50]:
def compute_similarity(distances, dropout=0.2):

    distances = distances.flatten()
    # Apply softmax to the negative distances
    similarities = np.exp(-distances)
    softmax_scores = similarities / np.sum(similarities)

    # Apply dropout (randomly zero out some softmax scores)
    dropout_mask = np.random.binomial(1, 1 - dropout, size=softmax_scores.shape)
    dropped_softmax_scores = softmax_scores * dropout_mask

    # Renormilizing softmax scores so that they sum to 1 again.
    final_softmax = dropped_softmax_scores / np.sum(dropped_softmax_scores)
    
    # how to weigh in the final embedding? -> weigh emebedding more if they are closer in vector space.
    return final_softmax, distances


## Mask To Drop The Dropped Out Values


In [51]:
def filter_by_mask(arr1, arr2, arr3):
    """
    Removes elements from arr2 and arr3 where corresponding indices in arr1 are zero.
    """
    mask = arr1 != 0  # Create a boolean mask where arr1 is nonzero
    return arr1[mask], arr2[mask], arr3[mask]


## Linear Trasnform The Value Component and Correlation Features


In [52]:
class MLP_Wv(nn.Module):
    def __init__(self, input_dim):
        super(MLP_Wv, self).__init__()
        self.layer1 = nn.Linear(input_dim, 32)
        self.activation1 = nn.SiLU()

        self.layer2 = nn.Linear(32, 32)
        self.activation2 = nn.SiLU()
        self.dropout2 = nn.Dropout(p=0.2)

        self.layer3 = nn.Linear(32, 32)  # Fix: Output should be 32
        self.activation3 = nn.SiLU()  # Fix: Apply SiLU activation

    def forward(self, x):
        x = self.activation1(self.layer1(x))
        x = self.dropout2(self.activation2(self.layer2(x)))
        x = self.activation3(self.layer3(x))  # Fix: Apply activation & dropout
        return x


In [53]:
def compute_l1(distances):

    l1_dist = np.sqrt(distances)
    model = MLP_Wv(l1_dist.shape[0])
    # Convert input to tensor and pass it through the model
    l1_dist_tensor = torch.tensor(l1_dist, dtype=torch.float32)
    value_Wv = model(l1_dist_tensor).detach().numpy()

    return value_Wv


In [54]:
class MLP_Wy(nn.Module):
    """
    Instantiate MLP with input_dim=32 (from Wy)
    Expected output shape: (60,)
    """
    def __init__(self, input_dim):
        super(MLP_Wy, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)  
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 32)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)  # Shape (60, 1)
        return x.squeeze(1)  # Shape (60,)


In [55]:
def compute_wy(indices):

    y_i = train_y["isFraud"].loc[indices].values
    feature_target_tensor = torch.tensor(y_i, dtype=torch.long)

    # Define Wy: An embedding layer to map to 32-dim space
    embedding_dim = 32
    num_classes = 2  # Since input values are 0 or 1

    Wy = nn.Embedding(num_classes, embedding_dim)
    mlp = MLP_Wy(input_dim=embedding_dim)

    # Compute embeddings using Wy
    embeddings = Wy(feature_target_tensor)  # Shape: (60, 32)

    # Pass embeddings through MLP
    value_Wy = mlp(embeddings)  # Shape: (60,)

    return value_Wy

In [56]:
def compute_imp_features(indices):
    """
    Transform the features V86, V87, V79, and V94 using MLP_Wy
    """

    v_86 = train_df["V86"].loc[indices].values
    feature_target_tensor = torch.tensor(v_86, dtype=torch.float32).unsqueeze(-1)

    v_87 = train_df["V87"].loc[indices].values
    feature_target_tensor_1 = torch.tensor(v_87, dtype=torch.float32).unsqueeze(-1)

    v_79 = train_df["V79"].loc[indices].values
    feature_target_tensor_2 = torch.tensor(v_79, dtype=torch.float32).unsqueeze(-1)

    v_94 = train_df["V94"].loc[indices].values
    feature_target_tensor_3 = torch.tensor(v_94, dtype=torch.float32).unsqueeze(-1)

    # Convert to PyTorch tensor
    input_dim = 1  # Since each input is a single continuous value
    embedding_dim = 32
    W_feat = nn.Linear(input_dim, embedding_dim)
    mlp = MLP_Wy(input_dim=embedding_dim)

    # Process first variable
    embeddings_1 = W_feat(feature_target_tensor)
    feat_x1 = mlp(embeddings_1)
    feat_x1 = feat_x1.detach().numpy()
    # Process second variable
    embeddings_2 = W_feat(feature_target_tensor_1)
    feat_x2 = mlp(embeddings_2)
    feat_x2 = feat_x2.detach().numpy()
    # Process third variable
    embeddings_3 = W_feat(feature_target_tensor_2)
    feat_x3 = mlp(embeddings_3)
    feat_x3 = feat_x3.detach().numpy()
    # Process fourth variable
    embeddings_4 = W_feat(feature_target_tensor_3)
    feat_x4 = mlp(embeddings_4)
    feat_x4 = feat_x4.detach().numpy()

    return feat_x1, feat_x2, feat_x3, feat_x4

In [57]:
def compute_value(value_Wv, value_Wy):
    """
    Compute the dot product of value_Wv and value_Wy
    """    
    value_Wy_npy = value_Wy.detach().numpy()
    value = value_Wy_npy + value_Wv
    return value

#### Reshaping S to do S \* V


In [58]:
def compute_z_in(similarity, value):
    """
    z_in is the intergrated(weighted sum) result of value and similarity componenet
    """
    similarity = similarity.reshape(1, -1)

    numerator = np.sum(similarity @ value)  # Sum the weighted contributions (scalar)
    denominator = np.sum(similarity)        # Total sum of weights (scalar)
    z_in = numerator / denominator          # Weighted average as a single scala
    
    return z_in


### Process Samples for Training and Validation


In [59]:
# test_i = test_df.iloc[10].values #shape (182,)
# query_vector = test_embeddings[10] #shape (32,)
# distances, indices = search_faiss(query_vector) # both shape (120,) and flatten
# similarity, distances = compute_similarity(distances)
# similarity, indices, distances = filter_by_mask(similarity, indices, distances)
# value_Wv = compute_l1(distances) #shape (32,)
# value_Wy = compute_wy(indices) #shape (32,)


# feat_x_1, feat_x2, feat_x3, feat_x4 = compute_imp_features(indices=indices)
# #z_in
# f_z_1 = compute_z_in(similarity, feat_x_1)
# f_z_2 = compute_z_in(similarity, feat_x2)
# f_z_3 = compute_z_in(similarity, feat_x3)
# f_z_4 = compute_z_in(similarity, feat_x4)

# f_z_1, f_z_2, f_z_3, f_z_4

# value = compute_value(value_Wv, value_Wy)
# value.shape


# labels = test_y['isFraud']
# labels = torch.tensor(test_y['isFraud'].values, dtype=torch.float32)
# type(labels)

# type(test_df.iloc[34].values)

# def process_samples(test_df, test_embeddings):
#     """Processes all samples and returns input tensor and labels."""
#     input_list = []
#     labels = []


#     for i in range(len(test_df)):  # Process all samples
#         test_i = torch.tensor(test_df.iloc[i].values, dtype=torch.float32)  # shape (182,)
#         query_embedding = torch.tensor(test_embeddings[i], dtype=torch.float32)  # shape (32,)

#         distances, indices = search_faiss(query_embedding)
#         similarity, distances = compute_similarity(distances)
#         similarity, indices, distances = filter_by_mask(similarity, indices, distances)
#         value_Wv = compute_l1(distances)  # shape (32,)
#         value_Wy = compute_wy(indices)  # shape (32,)
#         value = compute_value(value_Wv, value_Wy)
#         z_in = compute_z_in(similarity, value)  # Scalar (single value)

#         feat_x_1, feat_x2, feat_x3, feat_x4 = compute_imp_features(indices=indices) #v86 feature
#         f_z_in_1 = compute_z_in(similarity, feat_x_1)

#         f_z_in_2 = compute_z_in(similarity, feat_x2)

#         f_z_in_3 = compute_z_in(similarity, feat_x3)

#         f_z_in_4 = compute_z_in(similarity, feat_x4)

#         # Append only test_i (input_emb) and z_in (weighted_avg), not query_embedding
#         z_in = torch.tensor(z_in, dtype=torch.float32)
#         f_z_in_1 = torch.tensor(f_z_in_1, dtype=torch.float32)
#         f_z_in_2 = torch.tensor(f_z_in_2, dtype=torch.float32)
#         f_z_in_3 = torch.tensor(f_z_in_3, dtype=torch.float32)
#         f_z_in_4 = torch.tensor(f_z_in_4, dtype=torch.float32)

#         input_list.append((test_i, z_in, f_z_in_1, f_z_in_2, f_z_in_3, f_z_in_4))

#         labels.append(torch.tensor(test_y.iloc[i]['isFraud'], dtype=torch.float32))  # Assuming label is in df


#     return input_list, labels


In [60]:
def process_samples(train_df, train_embeddings, batch_size=1000, delay=0.5):
    """
    Processes samples in batches to prevent memory overflow.
    """
    input_list = []
    labels = []

    total_samples = len(train_df)
    num_batches = (
        total_samples + batch_size - 1
    ) // batch_size  # Compute number of batches

    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = min((batch_idx + 1) * batch_size, total_samples)

        batch_inputs = []
        batch_labels = []

        for i in range(start_idx, end_idx):  # Process each sample in batch
            test_i = torch.tensor(train_df.iloc[i].values, dtype=torch.float32)  # shape (182,)
            query_embedding = torch.tensor(train_embeddings[i], dtype=torch.float32)  # shape (32,)

            distances, indices = search_faiss(query_embedding)
            similarity, distances = compute_similarity(distances)
            similarity, indices, distances = filter_by_mask(similarity, indices, distances)
            value_Wv = compute_l1(distances)  # shape (32,)
            value_Wy = compute_wy(indices)  # shape (32,)
            value = compute_value(value_Wv, value_Wy)
            z_in = compute_z_in(similarity, value)  # Scalar (single value)

            feat_x_1, feat_x2, feat_x3, feat_x4 = compute_imp_features(
                indices=indices
            )
            f_z_in_1 = compute_z_in(similarity, feat_x_1)
            f_z_in_2 = compute_z_in(similarity, feat_x2)
            f_z_in_3 = compute_z_in(similarity, feat_x3)
            f_z_in_4 = compute_z_in(similarity, feat_x4)

            # Convert to tensors
            z_in = torch.tensor(z_in, dtype=torch.float32)
            f_z_in_1 = torch.tensor(f_z_in_1, dtype=torch.float32)
            f_z_in_2 = torch.tensor(f_z_in_2, dtype=torch.float32)
            f_z_in_3 = torch.tensor(f_z_in_3, dtype=torch.float32)
            f_z_in_4 = torch.tensor(f_z_in_4, dtype=torch.float32)

            batch_inputs.append((test_i, z_in, f_z_in_1, f_z_in_2, f_z_in_3, f_z_in_4))
            batch_labels.append(
                torch.tensor(train_y.iloc[i]["isFraud"], dtype=torch.float32)
            )

        input_list.extend(batch_inputs)
        labels.extend(batch_labels)

        time.sleep(delay)  # Add delay to avoid system overload

    return input_list, labels


In [61]:
def val_process_samples(val_df, val_embeddings, batch_size=1000, delay=0.5):
    """
    Processes samples in batches to prevent memory overflow.
    """
    input_list = []
    labels = []

    total_samples = len(val_df)
    num_batches = (
        total_samples + batch_size - 1
    ) // batch_size  # Compute number of batches

    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = min((batch_idx + 1) * batch_size, total_samples)

        batch_inputs = []
        batch_labels = []

        for i in range(start_idx, end_idx):  # Process each sample in batch
            test_i = torch.tensor(val_df.iloc[i].values, dtype=torch.float32)  # shape (182,)
            query_embedding = torch.tensor(val_embeddings[i], dtype=torch.float32)  # shape (32,)

            distances, indices = search_faiss(query_embedding)
            similarity, distances = compute_similarity(distances)
            similarity, indices, distances = filter_by_mask(similarity, indices, distances)
            value_Wv = compute_l1(distances)  # shape (32,)
            value_Wy = compute_wy(indices)  # shape (32,)
            value = compute_value(value_Wv, value_Wy)
            z_in = compute_z_in(similarity, value)  # Scalar (single value)

            feat_x_1, feat_x2, feat_x3, feat_x4 = compute_imp_features(
                indices=indices
            )  # v86 feature
            f_z_in_1 = compute_z_in(similarity, feat_x_1)
            f_z_in_2 = compute_z_in(similarity, feat_x2)
            f_z_in_3 = compute_z_in(similarity, feat_x3)
            f_z_in_4 = compute_z_in(similarity, feat_x4)

            # Convert to tensors
            z_in = torch.tensor(z_in, dtype=torch.float32)
            f_z_in_1 = torch.tensor(f_z_in_1, dtype=torch.float32)
            f_z_in_2 = torch.tensor(f_z_in_2, dtype=torch.float32)
            f_z_in_3 = torch.tensor(f_z_in_3, dtype=torch.float32)
            f_z_in_4 = torch.tensor(f_z_in_4, dtype=torch.float32)

            batch_inputs.append((test_i, z_in, f_z_in_1, f_z_in_2, f_z_in_3, f_z_in_4))
            batch_labels.append(torch.tensor(val_y.iloc[i]["isFraud"], dtype=torch.float32))

        input_list.extend(batch_inputs)
        labels.extend(batch_labels)

        time.sleep(delay)  # Add delay to avoid system overload

    return input_list, labels


## Model Training

In [ ]:
class Predictor(nn.Module):
    def __init__(self, input_emb_dim, hidden_dim=32, dropout_prob=0.2):
        super(Predictor, self).__init__()

        self.input_dim = input_emb_dim + 4  # Adding 2 for weighted_avg and Corr features

        self.block1 = nn.Sequential(
            nn.LayerNorm(self.input_dim),
            nn.Linear(self.input_dim, hidden_dim),
            nn.SiLU(),
            nn.Dropout(dropout_prob),
            nn.Linear(hidden_dim, hidden_dim),
        )

        self.block2 = nn.Sequential(
            nn.LayerNorm(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.SiLU(),
            nn.Dropout(dropout_prob),
            nn.Linear(hidden_dim, hidden_dim),
        )

        self.block3 = nn.Sequential(
            nn.LayerNorm(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.SiLU(),
            nn.Dropout(dropout_prob),
            nn.Linear(hidden_dim, hidden_dim),
        )

        self.output_layer = nn.Sequential(nn.Linear(hidden_dim, 1), nn.Sigmoid())

    def forward(self, input_emb, weighted_avg, f_z_in, f_z_in_2, f_z_in_3, f_z_in_4):
        # Ensure correct shape for scalar inputs
        weighted_avg = weighted_avg.unsqueeze(-1)
        
        f_z_in = f_z_in.unsqueeze(-1)
        f_z_in_2 = f_z_in_2.unsqueeze(-1)
        f_z_in_3 = f_z_in_3.unsqueeze(-1)
        f_z_in_4 = f_z_in_4.unsqueeze(-1)

        # Concatenate all inputs
        combined = torch.cat(
            [input_emb, weighted_avg, f_z_in, f_z_in_3, f_z_in_4], dim=-1
        )

        # Pass through MLP blocks
        x = self.block1(combined)
        x = self.block2(x)
        x = self.block3(x)
        x = self.output_layer(x)
        return x


In [63]:
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import average_precision_score
import torch

def train_model(test_df, test_embeddings, model, optimizer, criterion, batch_size=256, epochs=25, log_file="training_log.txt"):
    model.train()
    
    # Process data
    input_list, labels = process_samples(train_df, train_embeddings)
    val_input_list, val_labels = val_process_samples(val_df, val_embeddings)


    # Create DataLoader with `f_z_in`
    train_dataset = TensorDataset(
        torch.stack([item[0] for item in input_list]),  # test_i (input_emb)
        torch.stack([item[1] for item in input_list]),  # z_in (weighted_avg)
        torch.stack([item[2] for item in input_list]),  # f_z_in (new scalar input)
        torch.stack([item[3] for item in input_list]),  # f_z_in_2 (new scalar input)
        torch.stack([item[4] for item in input_list]),  # f_z_in_3 (new scalar input)
        torch.stack([item[5] for item in input_list]),  # f_z_in_4 (new scalar input)
        torch.stack(labels)
    )

    val_dataset = TensorDataset(
        torch.stack([item[0] for item in val_input_list]),  # test_i (input_emb)
        torch.stack([item[1] for item in val_input_list]),  # z_in (weighted_avg)
        torch.stack([item[2] for item in val_input_list]),  # f_z_in (new scalar input)
        torch.stack([item[3] for item in val_input_list]),  # f_z_in_2 (new scalar input)
        torch.stack([item[4] for item in val_input_list]),  # f_z_in_3 (new scalar input)
        torch.stack([item[5] for item in val_input_list]),  # f_z_in_4 (new scalar input)
        torch.stack(val_labels)
    )
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Training loop
    for epoch in range(epochs):
        epoch_loss = 0  # Track loss for each epoch
        all_targets = []
        all_outputs = []

        for batch in train_dataloader:
            input_emb, z_in, f_z_in, f_z_in_2, f_z_in_3, f_z_in_4, target = batch
            target = target.unsqueeze(-1)  # Make target shape (batch_size, 1)

            optimizer.zero_grad()
            output = model(input_emb, z_in, f_z_in, f_z_in_2, f_z_in_3, f_z_in_4)  # Pass f_z_in to model

            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()  # Accumulate loss for epoch
            
            # Collect predictions & targets for AUCPR
            all_outputs.append(output.detach().cpu())  # Move to CPU to avoid memory issues
            all_targets.append(target.detach().cpu())

        # Compute AUCPR at the end of the epoch
        all_outputs = torch.cat(all_outputs).numpy()
        all_targets = torch.cat(all_targets).numpy()
        aucpr = average_precision_score(all_targets, all_outputs)

        # Validation Step
        model.eval()
        val_loss = 0
        all_val_targets = []
        all_val_outputs = []

        with torch.no_grad():  # No gradient computation for validation
            for batch in val_dataloader:
                input_emb, z_in, f_z_in, f_z_in_2, f_z_in_3, f_z_in_4, target = batch
                target = target.unsqueeze(-1)

                output = model(input_emb, z_in, f_z_in, f_z_in_2, f_z_in_3, f_z_in_4)
                loss = criterion(output, target)

                val_loss += loss.item()
                
                all_val_outputs.append(output.cpu())
                all_val_targets.append(target.cpu())

        # Compute AUCPR for validation set
        all_val_outputs = torch.cat(all_val_outputs).numpy()
        all_val_targets = torch.cat(all_val_targets).numpy()
        val_aucpr = average_precision_score(all_val_targets, all_val_outputs)

        print(f"Epoch [{epoch+1}/{epochs}], "
              f"Train Loss: {epoch_loss/len(train_dataloader):.4f}, Train AUCPR: {aucpr:.4f}, "
              f"Val Loss: {val_loss/len(val_dataloader):.4f}, Val AUCPR: {val_aucpr:.4f}")

# Model, optimizer, and loss function
input_emb_dim = 182  # Assuming this based on test_df features
model = Predictor(input_emb_dim=input_emb_dim)
optimizer = optim.AdamW(model.parameters(), lr=0.0001)
criterion = nn.BCELoss()

train_model(train_df, train_embeddings, model, optimizer, criterion, epochs=100)


Epoch [1/100], Train Loss: 0.2887, Train AUCPR: 0.0381, Val Loss: 0.1387, Val AUCPR: 0.1668
Epoch [2/100], Train Loss: 0.1351, Train AUCPR: 0.1887, Val Loss: 0.1297, Val AUCPR: 0.2511
Epoch [3/100], Train Loss: 0.1284, Train AUCPR: 0.2509, Val Loss: 0.1216, Val AUCPR: 0.3021
Epoch [4/100], Train Loss: 0.1214, Train AUCPR: 0.2946, Val Loss: 0.1171, Val AUCPR: 0.3263
Epoch [5/100], Train Loss: 0.1178, Train AUCPR: 0.3179, Val Loss: 0.1151, Val AUCPR: 0.3354
Epoch [6/100], Train Loss: 0.1157, Train AUCPR: 0.3320, Val Loss: 0.1140, Val AUCPR: 0.3405
Epoch [7/100], Train Loss: 0.1145, Train AUCPR: 0.3415, Val Loss: 0.1133, Val AUCPR: 0.3441
Epoch [8/100], Train Loss: 0.1136, Train AUCPR: 0.3490, Val Loss: 0.1128, Val AUCPR: 0.3471
Epoch [9/100], Train Loss: 0.1128, Train AUCPR: 0.3553, Val Loss: 0.1124, Val AUCPR: 0.3506
Epoch [10/100], Train Loss: 0.1121, Train AUCPR: 0.3609, Val Loss: 0.1121, Val AUCPR: 0.3530
Epoch [11/100], Train Loss: 0.1115, Train AUCPR: 0.3660, Val Loss: 0.1118, Val 

### Inference

In [66]:
def test_process_samples(test_df, test_embeddings, batch_size=1000, delay=0.5):
    """
    Processes samples in batches to prevent memory overflow.
    """
    input_list = []
    labels = []

    total_samples = len(test_df)
    num_batches = (
        total_samples + batch_size - 1
    ) // batch_size  # Compute number of batches

    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = min((batch_idx + 1) * batch_size, total_samples)

        batch_inputs = []
        batch_labels = []

        for i in range(start_idx, end_idx):  # Process each sample in batch
            test_i = torch.tensor(test_df.iloc[i].values, dtype=torch.float32)  # shape (182,)
            query_embedding = torch.tensor(test_embeddings[i], dtype=torch.float32)  # shape (32,)

            distances, indices = search_faiss(query_embedding)
            similarity, distances = compute_similarity(distances)
            similarity, indices, distances = filter_by_mask(similarity, indices, distances)
            value_Wv = compute_l1(distances)  # shape (32,)
            value_Wy = compute_wy(indices)  # shape (32,)
            value = compute_value(value_Wv, value_Wy)
            z_in = compute_z_in(similarity, value)  # Scalar (single value)

            feat_x_1, feat_x2, feat_x3, feat_x4 = compute_imp_features(
                indices=indices
            )  # v86 feature
            f_z_in_1 = compute_z_in(similarity, feat_x_1)
            f_z_in_2 = compute_z_in(similarity, feat_x2)
            f_z_in_3 = compute_z_in(similarity, feat_x3)
            f_z_in_4 = compute_z_in(similarity, feat_x4)

            # Convert to tensors
            z_in = torch.tensor(z_in, dtype=torch.float32)
            f_z_in_1 = torch.tensor(f_z_in_1, dtype=torch.float32)
            f_z_in_2 = torch.tensor(f_z_in_2, dtype=torch.float32)
            f_z_in_3 = torch.tensor(f_z_in_3, dtype=torch.float32)
            f_z_in_4 = torch.tensor(f_z_in_4, dtype=torch.float32)

            batch_inputs.append((test_i, z_in, f_z_in_1, f_z_in_2, f_z_in_3, f_z_in_4))
            batch_labels.append(torch.tensor(test_y.iloc[i]["isFraud"], dtype=torch.float32))

        input_list.extend(batch_inputs)
        labels.extend(batch_labels)

        time.sleep(delay)  # Add delay to avoid system overload

    return input_list, labels


In [69]:
import torch
from torch.utils.data import DataLoader, TensorDataset

def inference(model, test_df, test_embeddings, batch_size=256):
    """
    Perform inference on test data using a trained model.
    
    Args:
        model (nn.Module): Trained model.
        test_df (pd.DataFrame): Your test dataframe.
        test_embeddings (np.ndarray or torch.Tensor): Precomputed embeddings for test.
        batch_size (int): Batch size for inference.

    Returns:
        all_outputs (np.ndarray): Model outputs (probabilities).
        all_labels (np.ndarray): Ground truth labels.
    """
    test_input_list, test_input_labels = test_process_samples(test_df=test_df, test_embeddings=test_embeddings)

    model.eval()

    test_dataset = TensorDataset(
        torch.stack([item[0] for item in test_input_list]),  # test_i (input_emb)
        torch.stack([item[1] for item in test_input_list]),  # z_in (weighted_avg)
        torch.stack([item[2] for item in test_input_list]),  # f_z_in (new scalar input)
        torch.stack([item[3] for item in test_input_list]),  # f_z_in_2 (new scalar input)
        torch.stack([item[4] for item in test_input_list]),  # f_z_in_3 (new scalar input)
        torch.stack([item[5] for item in test_input_list]),  # f_z_in_4 (new scalar input)
        torch.stack(test_input_labels)
    )
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Lists to store outputs and labels
    all_outputs = []
    all_labels = []

    # No gradient updates during inference
    with torch.no_grad():
        for batch in test_dataloader:
            input_emb, z_in, f1, f2, f3, f4, labels = batch
            outputs = model(input_emb, z_in, f1, f2, f3, f4)
            
            # Move outputs and labels to CPU, then store
            all_outputs.append(outputs.cpu())
            all_labels.append(labels.cpu())

    # Concatenate everything into a single array/tensor
    all_outputs = torch.cat(all_outputs).numpy()
    all_labels = torch.cat(all_labels).numpy()

    return all_outputs, all_labels

all_test_outputs, all_test_labels = inference(model, test_df, test_embeddings, batch_size=256)

# Optionally, compute metrics (e.g., AUCPR) on your test set:
from sklearn.metrics import average_precision_score
test_aucpr = average_precision_score(all_test_labels, all_test_outputs)
print("Test AUCPR:", test_aucpr)

# Or create a final binary prediction if desired:
binary_predictions = (all_test_outputs >= 0.5).astype(int)


Test AUCPR: 0.4140655186866458


In [70]:
import numpy as np

# Count occurrences of each unique value (0s and 1s)
unique_values, counts = np.unique(binary_predictions, return_counts=True)

# Print results
for value, count in zip(unique_values, counts):
    print(f"Value {value}: {count} occurrences")


Value 0: 23258 occurrences
Value 1: 364 occurrences
